In [1]:
import numpy as np
import numba
from numba import jit

### The @jit decorator

In [35]:
# a function that does multiple matrix multiplication
@jit(nopython=True)
def matrix_multiplication(A, x):
    b = np.empty(shape=(x.shape[0],1), dtype=np.float64)
    for i in range(x.shape[0]):
        b[i] = np.dot(A[i,:], x)
    return b

In [36]:
A = np.random.rand(10, 10)
x = np.random.rand(10, 1)
a_complicated_function(A,x)

array([[2.79814834],
       [2.60367089],
       [3.64898388],
       [2.46134003],
       [2.20620761],
       [2.35963373],
       [2.78546053],
       [1.56009935],
       [3.57130984],
       [2.58911908]])

In [37]:
%timeit matrix_multiplication(A,x)

11.4 µs ± 7.34 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%timeit matrix_multiplication.py_func(A,x)

35.5 µs ± 3.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [59]:
# Calculate percentage relative error
def numpy_re(x, true):
    return np.abs(((x - true)/true))*100

numba_re = jit(nopython=True)(numpy_re)

In [60]:
numba_re(x, 0.66)

array([[56.99126084],
       [40.1849741 ],
       [14.89169085],
       [84.30874992],
       [ 9.60118043],
       [ 7.56524974],
       [33.54230711],
       [35.38826731],
       [65.90336653],
       [20.51053746]])

In [61]:
%timeit numpy_re(x, 0.66)
%timeit numba_re(x, 0.66)

2.61 µs ± 118 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
687 ns ± 40 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Inline parallelization with multi-threading

In [39]:
SQRT_2PI = np.sqrt(2 * np.pi)

@jit(nopython=True, parallel=True)
def normals(x, means, sds):
    n = means.shape[0]
    result = np.exp(-0.5*((x - means)/sds)**2)
    return (1 / (sds * np.sqrt(2*np.pi))) * result

In [48]:
means = np.random.uniform(-1,1, size=10**8)
sds = np.random.uniform(0.1, 0.2, size=10**8)

normals(0.6, means, sds)

array([4.38580749e-01, 2.79653674e-12, 1.16879868e+00, ...,
       3.01820734e-01, 1.60402426e+00, 2.64271255e+00])

In [49]:
normals_deco_nothread = jit(nopython=True)(normals.py_func)
%timeit normals_deco_nothread(0.6, means, sds)

3.24 s ± 757 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
%timeit normals.py_func(0.6, means, sds)

7.38 s ± 759 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
%timeit normals(0.6, means, sds)

933 ms ± 155 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
